# Joypy
## Joy in Python

This implementation is meant as a tool for exploring the programming model and method of Joy.  Python seems like a great implementation language for Joy for several reasons.

We can lean on the Python immutable types for our basic semantics and types: ints, floats, strings, and tuples, which enforces functional purity.  We get garbage collection for free.  Compilation via Cython.  Glue language with loads of libraries.

### [Read-Eval-Print Loop (REPL)](https://en.wikipedia.org/wiki/Read%E2%80%93eval%E2%80%93print_loop)
The main way to interact with the Joy interpreter is through a simple [REPL](https://en.wikipedia.org/wiki/Read%E2%80%93eval%E2%80%93print_loop) that you start by running the package:

    $ python -m joy
    Joypy - Copyright © 2017 Simon Forman
    This program comes with ABSOLUTELY NO WARRANTY; for details type "warranty".
    This is free software, and you are welcome to redistribute it
    under certain conditions; type "sharing" for details.
    Type "words" to see a list of all words, and "[<name>] help" to print the
    docs for a word.


     <-top

    joy? _

The `<-top` marker points to the top of the (initially empty) stack.  You can enter Joy notation at the prompt and a [trace of evaluation](#The-TracePrinter.) will be printed followed by the stack and prompt again:

    joy? 23 sqr 18 +
           . 23 sqr 18 +
        23 . sqr 18 +
        23 . dup mul 18 +
     23 23 . mul 18 +
       529 . 18 +
    529 18 . +
       547 . 

    547 <-top

    joy? 


# Stacks (aka list, quote, sequence, etc.)

In Joy, in addition to the types Boolean, integer, float, and string, there is a single sequence type represented by enclosing a sequence of terms in brackets `[...]`.  This sequence type is used to represent both the stack and the expression.  It is a [cons list](https://en.wikipedia.org/wiki/Cons#Lists) made from Python tuples.

In [1]:
import inspect
import joy.utils.stack


print inspect.getdoc(joy.utils.stack)

§ Stack


When talking about Joy we use the terms "stack", "list", "sequence" and
"aggregate" to mean the same thing: a simple datatype that permits
certain operations such as iterating and pushing and popping values from
(at least) one end.

We use the venerable two-tuple recursive form of sequences where the
empty tuple () is the empty stack and (head, rest) gives the recursive
form of a stack with one or more items on it.

  ()
  (1, ())
  (2, (1, ()))
  (3, (2, (1, ())))
  ...

And so on.


We have two very simple functions to build up a stack from a Python
iterable and also to iterate through a stack and yield its items
one-by-one in order, and two functions to generate string representations
of stacks:

  list_to_stack()

  iter_stack()

  expression_to_string()  (prints left-to-right)

  stack_to_string()  (prints right-to-left)


A word about the stack data structure.

Python has very nice "tuple packing and unpacking" in its syntax which
means we can directly "unpack" the expe

### The utility functions maintain order.
The 0th item in the list will be on the top of the stack and *vise versa*.

In [2]:
joy.utils.stack.list_to_stack([1, 2, 3])

(1, (2, (3, ())))

In [3]:
list(joy.utils.stack.iter_stack((1, (2, (3, ())))))

[1, 2, 3]

This requires reversing the sequence (or iterating backwards) otherwise:

In [4]:
stack = ()

for n in [1, 2, 3]:
    stack = n, stack

print stack
print list(joy.utils.stack.iter_stack(stack))

(3, (2, (1, ())))
[3, 2, 1]


### Purely Functional Datastructures.
Because Joy lists are made out of Python tuples they are immutable, so all Joy datastructures are *[purely functional](https://en.wikipedia.org/wiki/Purely_functional_data_structure)*.

# The `joy()` function.
## An Interpreter
The `joy()` function is extrememly simple.  It accepts a stack, an expression, and a dictionary, and it iterates through the expression putting values onto the stack and delegating execution to functions it looks up in the dictionary.

Each function is passed the stack, expression, and dictionary and returns them.  Whatever the function returns becomes the new stack, expression, and dictionary.  (The dictionary is passed to enable e.g. writing words that let you enter new words into the dictionary at runtime, which nothing does yet and may be a bad idea, and the `help` command.)

In [5]:
import joy.joy

print inspect.getsource(joy.joy.joy)

def joy(stack, expression, dictionary, viewer=None):
  '''
  Evaluate the Joy expression on the stack.
  '''
  while expression:

    if viewer: viewer(stack, expression)

    term, expression = expression
    if isinstance(term, Symbol):
      term = dictionary[term]
      stack, expression, dictionary = term(stack, expression, dictionary)
    else:
      stack = term, stack

  if viewer: viewer(stack, expression)
  return stack, expression, dictionary



### View function
The `joy()` function accepts a "viewer" function which it calls on each iteration passing the current stack and expression just before evaluation.  This can be used for tracing, breakpoints, retrying after exceptions, or interrupting an evaluation and saving to disk or sending over the network to resume later.  The stack and expression together contain all the state of the computation at each step.

### The `TracePrinter`.

A `viewer` records each step of the evaluation of a Joy program.  The `TracePrinter` has a facility for printing out a trace of the evaluation, one line per step.  Each step is aligned to the current interpreter position, signified by a period separating the stack on the left from the pending expression ("continuation") on the right.

### [Continuation-Passing Style](https://en.wikipedia.org/wiki/Continuation-passing_style)
One day I thought, What happens if you rewrite Joy to use [CSP](https://en.wikipedia.org/wiki/Continuation-passing_style)?  I made all the functions accept and return the expression as well as the stack and found that all the combinators could be rewritten to work by modifying the expression rather than making recursive calls to the `joy()` function.

# Parser

In [6]:
import joy.parser

print inspect.getdoc(joy.parser)

§ Converting text to a joy expression.

This module exports a single function:

  text_to_expression(text)

As well as a single Symbol class and a single Exception type:

  ParseError

When supplied with a string this function returns a Python datastructure
that represents the Joy datastructure described by the text expression.
Any unbalanced square brackets will raise a ParseError.


The parser is extremely simple, the undocumented `re.Scanner` class does most of the tokenizing work and then you just build the tuple structure out of the tokens.  There's no Abstract Syntax Tree or anything like that.

In [7]:
print inspect.getsource(joy.parser._parse)

def _parse(tokens):
  '''
  Return a stack/list expression of the tokens.
  '''
  frame = []
  stack = []
  for tok in tokens:
    if tok == '[':
      stack.append(frame)
      frame = []
      stack[-1].append(frame)
    elif tok == ']':
      try:
        frame = stack.pop()
      except IndexError:
        raise ParseError('One or more extra closing brackets.')
      frame[-1] = list_to_stack(frame[-1])
    else:
      frame.append(tok)
  if stack:
    raise ParseError('One or more unclosed brackets.')
  return list_to_stack(frame)



That's pretty much all there is to it.

In [8]:
joy.parser.text_to_expression('1 2 3 4 5')  # A simple sequence.

(1, (2, (3, (4, (5, ())))))

In [9]:
joy.parser.text_to_expression('[1 2 3] 4 5')  # Three items, the first is a list with three items

((1, (2, (3, ()))), (4, (5, ())))

In [10]:
joy.parser.text_to_expression('1 23 ["four" [-5.0] cons] 8888')  # A mixed bag. cons is
                                                                 # a Symbol, no lookup at
                                                                 # parse-time.  Haiku docs.

(1, (23, (('four', ((-5.0, ()), (cons, ()))), (8888, ()))))

In [11]:
joy.parser.text_to_expression('[][][][][]')  # Five empty lists.

((), ((), ((), ((), ((), ())))))

In [12]:
joy.parser.text_to_expression('[[[[[]]]]]')  # Five nested lists.

((((((), ()), ()), ()), ()), ())

# Library
The Joy library of functions (aka commands, or "words" after Forth usage) encapsulates all the actual functionality (no pun intended) of the Joy system.  There are simple functions such as addition `add` (or `+`, the library module supports aliases), and combinators which provide control-flow and higher-order operations.

In [13]:
import joy.library

print ' '.join(sorted(joy.library.initialize()))

!= % & * *fraction *fraction0 + ++ - -- / < << <= <> = > >= >> ? ^ add anamorphism and app1 app2 app3 average b binary branch choice clear cleave concat cons dinfrirst dip dipd dipdd disenstacken div down_to_zero dudipd dup dupd dupdip enstacken eq first flatten floordiv gcd ge genrec getitem gt help i id ifte infra le least_fraction loop lshift lt map min mod modulus mul ne neg not nullary or over pam parse pm pop popd popdd popop pow pred primrec product quoted range range_to_zero rem remainder remove rest reverse roll< roll> rolldown rollup rshift run second select sharing shunt size sqr sqrt stack step sub succ sum swaack swap swoncat swons ternary third times truediv truthy tuck unary uncons unit unquoted unstack void warranty while words x xor zip •


Many of the functions are defined in Python, like `dip`:

In [14]:
print inspect.getsource(joy.library.dip)

def dip(stack, expression, dictionary):
  (quote, (x, stack)) = stack
  expression = x, expression
  return stack, pushback(quote, expression), dictionary



Some functions are defined in equations in terms of other functions.  When the interpreter executes a definition function that function just pushes its body expression onto the pending expression (the continuation) and returns control to the interpreter.

In [15]:
print joy.library.definitions

second == rest first
third == rest rest first
product == 1 swap [*] step
swons == swap cons
swoncat == swap concat
flatten == [] swap [concat] step
unit == [] cons
quoted == [unit] dip
unquoted == [i] dip
enstacken == stack [clear] dip
disenstacken == ? [uncons ?] loop pop
? == dup truthy
dinfrirst == dip infra first
nullary == [stack] dinfrirst
unary == [stack [pop] dip] dinfrirst
binary == [stack [popop] dip] dinfrirst
ternary == [stack [popop pop] dip] dinfrirst
pam == [i] map
run == [] swap infra
sqr == dup mul
size == 0 swap [pop ++] step
cleave == [i] app2 [popd] dip
average == [sum 1.0 *] [size] cleave /
gcd == 1 [tuck modulus dup 0 >] loop pop
least_fraction == dup [gcd] infra [div] concat map
*fraction == [uncons] dip uncons [swap] dip concat [*] infra [*] dip cons
*fraction0 == concat [[swap] dip * [*] dip] infra
down_to_zero == [0 >] [dup --] while
range_to_zero == unit [down_to_zero] infra
anamorphism == [pop []] swap [dip swons] genrec
range == [0 <=] [1 - dup] anamorphism

Currently, there's no function to add new definitions to the dictionary from "within" Joy code itself.  Adding new definitions remains a meta-interpreter action.  You have to do it yourself, in Python, and wash your hands afterward.

It would be simple enough to define one, but it would open the door to *name binding* and break the idea that all state is captured in the stack and expression.  There's an implicit *standard dictionary* that defines the actual semantics of the syntactic stack and expression datastructures (which only contain symbols, not the actual functions.  Pickle some and see for yourself.)

#### "There should be only one."

Which brings me to talking about one of my hopes and dreams for this notation:  "There should be only one."  What I mean is that there should be one universal standard dictionary of commands, and all bespoke work done in a UI for purposes takes place by direct interaction and macros.  There would be a *Grand Refactoring* biannually (two years, not six months, that's semi-annually) where any new definitions factored out of the usage and macros of the previous time, along with new algorithms and such, were entered into the dictionary and posted to e.g. IPFS.

Code should not burgeon wildly, as it does today.  The variety of code should map more-or-less to the well-factored variety of human computably-solvable problems.  There shouldn't be dozens of chat apps, JS frameworks, programming languages.  It's a waste of time, a [fractal "thundering herd" attack](https://en.wikipedia.org/wiki/Thundering_herd_problem) on human mentality.

#### Literary Code Library

If you read over the other notebooks you'll see that developing code in Joy is a lot like doing simple mathematics, and the descriptions of the code resemble math papers.  The code also works the first time, no bugs.  If you have any experience programming at all, you are probably skeptical, as I was, but it seems to work: deriving code mathematically seems to lead to fewer errors.

But my point now is that this great ratio of textual explanation to wind up with code that consists of a few equations and could fit on an index card is highly desirable.  Less code has fewer errors.  The structure of Joy engenders a kind of thinking that seems to be very effective for developing structured processes.

There seems to be an elegance and power to the notation.
